In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier

# Эмуляция данных
np.random.seed(42)
n_questions = 100
n_respondents = 10
n_answers = n_questions * n_respondents

# Генерация текста вопросов и ответов
questions_text = [f"Вопрос {i}" for i in range(n_questions)]
respondents_id = np.repeat(np.arange(n_respondents), n_questions)
questions_id = np.tile(np.arange(n_questions), n_respondents)
correct_answers = np.random.binomial(n=1, p=0.5, size=n_answers)

# Векторизация вопросов
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(questions_text)

# Вычисление косинусного расстояния и кластеризация вопросов
similarity_matrix = cosine_similarity(question_vectors)
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(similarity_matrix)

# Применение PCA
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(question_vectors.toarray())

# Подготовка обучающих данных
data = pd.DataFrame({
    'RespondentID': respondents_id,
    'QuestionID': questions_id,
    'Correct': correct_answers,
    'Cluster': clusters[questions_id],
    'PC1': reduced_vectors[questions_id, 0],
    'PC2': reduced_vectors[questions_id, 1]
})

# Инициализация оценки способностей с использованием первых 10 вопросов
initial_questions = data.groupby('Cluster').head(10).index
X_initial = data.loc[initial_questions, ['Cluster', 'PC1', 'PC2']]
y_initial = data.loc[initial_questions, 'Correct']

# Обучение модели CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X_initial, y_initial, test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=100, learning_rate=1, depth=2, loss_function='Logloss', verbose=False)
model.fit(X_train, y_train)

# Оценка модели
accuracy = model.score(X_test, y_test)

from catboost import CatBoostClassifier

def update_ability_estimate(model, X, y_actual, current_ability):
    probabilities = model.predict_proba(X)[:, 1]
    error = probabilities - y_actual
    mean_error = np.mean(error)
    updated_ability = current_ability - mean_error * 0.1
    return updated_ability

def select_next_question_catboost(model, questions, current_ability, n_questions=1):
    # Подготовка DataFrame для предсказания с учётом текущей способности
    # Пример работы без использования 'CurrentAbility', если он не включён в обученную модель
    probabilities = model.predict_proba(questions[['Cluster', 'PC1', 'PC2']])[:, 1]
    # Сортировка вопросов по убыванию вероятности правильного ответа
    selected_questions = np.argsort(-probabilities)[:n_questions]
    return selected_questions



# Имитация процесса динамического выбора вопросов
current_ability = 0.5  # Пример начальной оценки способностей
available_questions = np.setdiff1d(np.arange(n_questions), data.loc[initial_questions, 'QuestionID'])


for i in range(10):  # выбор ещё 10 вопросов динамически
    questions_df = data.loc[data['QuestionID'].isin(available_questions), ['Cluster', 'PC1', 'PC2']]
    # Получаем индексы для предсказаний
    selected_indexes = select_next_question_catboost(model, questions_df, current_ability, 1)
    next_question_id = questions_df.iloc[selected_indexes].index[0]  # Получаем индекс следующего вопроса в DataFrame
    next_question_global_id = data.loc[next_question_id, 'QuestionID']  # Получаем глобальный ID следующего вопроса
    print(f"Выбран вопрос с ID: {next_question_global_id}")  # Вывод ID следующего вопроса
    print(f"Текст вопроса: {questions_text[next_question_global_id]}")  # Вывод текста вопроса
    
    available_questions = np.setdiff1d(available_questions, [next_question_global_id])
    
    X_new = data.loc[next_question_id, ['Cluster', 'PC1', 'PC2']].to_frame().T
    y_new = data.loc[next_question_id, 'Correct']
    
    # Расчет вероятности правильного ответа на выбранный вопрос
    probability_of_correct_answer = model.predict_proba(X_new)[:, 1][0]
    print(f"Вероятность правильного ответа на выбранный вопрос: {probability_of_correct_answer:.2f}")
    
    current_ability = update_ability_estimate(model, X_new, np.array([y_new]), current_ability)
    print(f"Обновлённая оценка способностей: {current_ability:.2f}")  # Вывод обновлённой оценки способностей

    if len(available_questions) == 0:
        break

Выбран вопрос с ID: 40
Текст вопроса: Вопрос 40
Вероятность правильного ответа на выбранный вопрос: 0.87
Обновлённая оценка способностей: 0.41
Выбран вопрос с ID: 82
Текст вопроса: Вопрос 82
Вероятность правильного ответа на выбранный вопрос: 0.87
Обновлённая оценка способностей: 0.33
Выбран вопрос с ID: 76
Текст вопроса: Вопрос 76
Вероятность правильного ответа на выбранный вопрос: 0.87
Обновлённая оценка способностей: 0.34
Выбран вопрос с ID: 96
Текст вопроса: Вопрос 96
Вероятность правильного ответа на выбранный вопрос: 0.87
Обновлённая оценка способностей: 0.25
Выбран вопрос с ID: 25
Текст вопроса: Вопрос 25
Вероятность правильного ответа на выбранный вопрос: 0.87
Обновлённая оценка способностей: 0.16
Выбран вопрос с ID: 64
Текст вопроса: Вопрос 64
Вероятность правильного ответа на выбранный вопрос: 0.87
Обновлённая оценка способностей: 0.18
Выбран вопрос с ID: 93
Текст вопроса: Вопрос 93
Вероятность правильного ответа на выбранный вопрос: 0.87
Обновлённая оценка способностей: 0.19

In [ ]:
!pip install openpyxl

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

data = pd.read_excel('questions_dataset.xlsx')

encoder = OneHotEncoder(sparse=False)
modality_encoded = encoder.fit_transform(data[['модальность_вопроса']])
modality_encoded_df = pd.DataFrame(modality_encoded, columns=encoder.get_feature_names_out(['модальность_вопроса']))

tfidf_vectorizer = TfidfVectorizer()
question_vectors = tfidf_vectorizer.fit_transform(data['текст_вопроса'])
question_vectors_df = pd.DataFrame(question_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

data_preprocessed = pd.concat([data.drop(['текст_вопроса', 'модальность_вопроса'], axis=1), modality_encoded_df], axis=1)
full_data_preprocessed = pd.concat([data_preprocessed, question_vectors_df], axis=1)
full_data_preprocessed['Правильность ответа'] = np.random.randint(0, 2, size=full_data_preprocessed.shape[0])

X = full_data_preprocessed.drop('Правильность ответа', axis=1)
y = full_data_preprocessed['Правильность ответа']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

def select_next_question(model, X, threshold=0.5):
    probabilities = model.predict_proba(X)[:, 1]
    max_prob_index = np.argmax(probabilities)
    if probabilities[max_prob_index] > threshold:
        return max_prob_index
    else:
        return None

X_questions = X_test
next_question_index = select_next_question(model, X_questions)
if next_question_index is not None:
    print(f"Следующий вопрос: {data['текст_вопроса'].iloc[next_question_index]}")
else:
    print("Нет подходящих вопросов.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 19.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Точность модели: 0.6666666666666666
Следующий вопрос: Что такое квадратный корень из 16?


In [ ]:
def assess_student_ability(data, questions_text, model, pca, vectorizer, kmeans, initial_questions):
    """
    Assess the student's ability by presenting questions and adjusting the difficulty based on their responses.
    
    :param data: DataFrame containing questions, their clusters, PCA components, and correct answers.
    :param questions_text: List containing text of the questions.
    :param model: Trained CatBoostClassifier model.
    :param pca: Trained PCA model.
    :param vectorizer: Trained TfidfVectorizer.
    :param kmeans: Trained KMeans model.
    :param initial_questions: List of initial questions to start the assessment.
    :return: Updated DataFrame marking nearby questions based on student's responses.
    """
    # Initialize
    current_ability = 0.5  # Example starting ability
    available_questions = np.setdiff1d(np.arange(len(questions_text)), initial_questions)
    threshold_distance = 0.1  # Predefined threshold distance in PCA space for marking nearby questions
    
    # Function to find nearby questions within the same cluster
    def find_nearby_questions(question_id, correct):
        cluster = data.loc[question_id, 'Cluster']
        pc1, pc2 = data.loc[question_id, ['PC1', 'PC2']]
        cluster_questions = data[data['Cluster'] == cluster]
        distances = np.sqrt((cluster_questions['PC1'] - pc1) ** 2 + (cluster_questions['PC2'] - pc2) ** 2)
        nearby_questions = cluster_questions[distances < threshold_distance].index
        data.loc[nearby_questions, 'Correct'] = correct
    
    # Dynamic question selection and adjustment based on responses
    for i in range(10):  # Dynamically choose 10 more questions
        questions_df = data.loc[data['QuestionID'].isin(available_questions), ['Cluster', 'PC1', 'PC2']]
        selected_indexes = select_next_question_catboost(model, questions_df, current_ability, 1)
        next_question_id = questions_df.iloc[selected_indexes].index[0]
        next_question_global_id = data.loc[next_question_id, 'QuestionID']
        
        # Simulate student's response (for demonstration, using actual correct answer)
        student_response = data.loc[next_question_id, 'Correct']
        
        # Mark nearby questions based on response
        find_nearby_questions(next_question_id, student_response)
        
        # Remove the answered question from available questions
        available_questions = np.setdiff1d(available_questions, [next_question_global_id])
        
        # Update student's ability based on response
        X_new = data.loc[next_question_id, ['Cluster', 'PC1', 'PC2']].to_frame().T
        current_ability = update_ability_estimate(model, X_new, np.array([student_response]), current_ability)
        
        if len(available_questions) == 0:
            break
    
    return data

# Call the function with the necessary parameters
assessed_data = assess_student_ability(data, questions_text, model, pca, vectorizer, kmeans, initial_questions)
assessed_data

NameError: name 'data' is not defined

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier

# Эмуляция данных
np.random.seed(42)
n_questions = 100
n_respondents = 10
n_answers = n_questions * n_respondents

# Генерация текста вопросов и ответов
questions_text = [f"Вопрос {i}" for i in range(n_questions)]
respondents_id = np.repeat(np.arange(n_respondents), n_questions)
questions_id = np.tile(np.arange(n_questions), n_respondents)
correct_answers = np.random.binomial(n=1, p=0.5, size=n_answers)

# Векторизация вопросов
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(questions_text)

# Вычисление косинусного расстояния и кластеризация вопросов
similarity_matrix = cosine_similarity(question_vectors)
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(similarity_matrix)

# Применение PCA
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(question_vectors.toarray())

# Подготовка обучающих данных
data = pd.DataFrame({
    'RespondentID': respondents_id,
    'QuestionID': questions_id,
    'Correct': correct_answers,
    'Cluster': clusters[questions_id],
    'PC1': reduced_vectors[questions_id, 0],
    'PC2': reduced_vectors[questions_id, 1]
})

# Инициализация оценки способностей с использованием первых 10 вопросов
initial_questions = data.groupby('Cluster').head(10).index
X_initial = data.loc[initial_questions, ['Cluster', 'PC1', 'PC2']]
y_initial = data.loc[initial_questions, 'Correct']

# Обучение модели CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X_initial, y_initial, test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=100, learning_rate=1, depth=2, loss_function='Logloss', verbose=False)
model.fit(X_train, y_train)

# Оценка модели
accuracy = model.score(X_test, y_test)

from catboost import CatBoostClassifier

def update_ability_estimate(model, X, y_actual, current_ability):
    probabilities = model.predict_proba(X)[:, 1]
    error = probabilities - y_actual
    mean_error = np.mean(error)
    updated_ability = current_ability - mean_error * 0.1
    return updated_ability

def select_next_question_catboost(model, questions, current_ability, n_questions=1):
    # Подготовка DataFrame для предсказания с учётом текущей способности
    # Пример работы без использования 'CurrentAbility', если он не включён в обученную модель
    probabilities = model.predict_proba(questions[['Cluster', 'PC1', 'PC2']])[:, 1]
    # Сортировка вопросов по убыванию вероятности правильного ответа
    selected_questions = np.argsort(-probabilities)[:n_questions]
    return selected_questions



# Имитация процесса динамического выбора вопросов
current_ability = 0.5  # Пример начальной оценки способностей
available_questions = np.setdiff1d(np.arange(n_questions), data.loc[initial_questions, 'QuestionID'])


for i in range(10):  # выбор ещё 10 вопросов динамически
    questions_df = data.loc[data['QuestionID'].isin(available_questions), ['Cluster', 'PC1', 'PC2']]
    # Получаем индексы для предсказаний
    selected_indexes = select_next_question_catboost(model, questions_df, current_ability, 1)
    next_question_id = questions_df.iloc[selected_indexes].index[0]  # Получаем индекс следующего вопроса в DataFrame
    next_question_global_id = data.loc[next_question_id, 'QuestionID']  # Получаем глобальный ID следующего вопроса
    print(f"Выбран вопрос с ID: {next_question_global_id}")  # Вывод ID следующего вопроса
    print(f"Текст вопроса: {questions_text[next_question_global_id]}")  # Вывод текста вопроса
    
    available_questions = np.setdiff1d(available_questions, [next_question_global_id])
    
    X_new = data.loc[next_question_id, ['Cluster', 'PC1', 'PC2']].to_frame().T
    y_new = data.loc[next_question_id, 'Correct']
    
    # Расчет вероятности правильного ответа на выбранный вопрос
    probability_of_correct_answer = model.predict_proba(X_new)[:, 1][0]
    print(f"Вероятность правильного ответа на выбранный вопрос: {probability_of_correct_answer:.2f}")
    
    current_ability = update_ability_estimate(model, X_new, np.array([y_new]), current_ability)
    print(f"Обновлённая оценка способностей: {current_ability:.2f}")  # Вывод обновлённой оценки способностей

    if len(available_questions) == 0:
        break

ModuleNotFoundError: No module named 'catboost'

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 12.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier

# Эмуляция данных
np.random.seed(42)
n_questions = 100
n_respondents = 10
n_answers = n_questions * n_respondents

# Генерация текста вопросов и ответов
questions_text = [f"Вопрос {i}" for i in range(n_questions)]
respondents_id = np.repeat(np.arange(n_respondents), n_questions)
questions_id = np.tile(np.arange(n_questions), n_respondents)
correct_answers = np.random.binomial(n=1, p=0.5, size=n_answers)

# Векторизация вопросов
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(questions_text)

# Вычисление косинусного расстояния и кластеризация вопросов
similarity_matrix = cosine_similarity(question_vectors)
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(similarity_matrix)

# Применение PCA
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(question_vectors.toarray())

# Подготовка обучающих данных
data = pd.DataFrame({
    'RespondentID': respondents_id,
    'QuestionID': questions_id,
    'Correct': correct_answers,
    'Cluster': clusters[questions_id],
    'PC1': reduced_vectors[questions_id, 0],
    'PC2': reduced_vectors[questions_id, 1]
})

# Инициализация оценки способностей с использованием первых 10 вопросов
initial_questions = data.groupby('Cluster').head(10).index
X_initial = data.loc[initial_questions, ['Cluster', 'PC1', 'PC2']]
y_initial = data.loc[initial_questions, 'Correct']

# Обучение модели CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X_initial, y_initial, test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=100, learning_rate=1, depth=2, loss_function='Logloss', verbose=False)
model.fit(X_train, y_train)

# Оценка модели
accuracy = model.score(X_test, y_test)

from catboost import CatBoostClassifier

def update_ability_estimate(model, X, y_actual, current_ability):
    probabilities = model.predict_proba(X)[:, 1]
    error = probabilities - y_actual
    mean_error = np.mean(error)
    updated_ability = current_ability - mean_error * 0.1
    return updated_ability

def select_next_question_catboost(model, questions, current_ability, n_questions=1):
    # Подготовка DataFrame для предсказания с учётом текущей способности
    # Пример работы без использования 'CurrentAbility', если он не включён в обученную модель
    probabilities = model.predict_proba(questions[['Cluster', 'PC1', 'PC2']])[:, 1]
    # Сортировка вопросов по убыванию вероятности правильного ответа
    selected_questions = np.argsort(-probabilities)[:n_questions]
    return selected_questions



# Имитация процесса динамического выбора вопросов
current_ability = 0.5  # Пример начальной оценки способностей
available_questions = np.setdiff1d(np.arange(n_questions), data.loc[initial_questions, 'QuestionID'])


for i in range(10):  # выбор ещё 10 вопросов динамически
    questions_df = data.loc[data['QuestionID'].isin(available_questions), ['Cluster', 'PC1', 'PC2']]
    # Получаем индексы для предсказаний
    selected_indexes = select_next_question_catboost(model, questions_df, current_ability, 1)
    next_question_id = questions_df.iloc[selected_indexes].index[0]  # Получаем индекс следующего вопроса в DataFrame
    next_question_global_id = data.loc[next_question_id, 'QuestionID']  # Получаем глобальный ID следующего вопроса
    print(f"Выбран вопрос с ID: {next_question_global_id}")  # Вывод ID следующего вопроса
    print(f"Текст вопроса: {questions_text[next_question_global_id]}")  # Вывод текста вопроса
    
    available_questions = np.setdiff1d(available_questions, [next_question_global_id])
    
    X_new = data.loc[next_question_id, ['Cluster', 'PC1', 'PC2']].to_frame().T
    y_new = data.loc[next_question_id, 'Correct']
    
    # Расчет вероятности правильного ответа на выбранный вопрос
    probability_of_correct_answer = model.predict_proba(X_new)[:, 1][0]
    print(f"Вероятность правильного ответа на выбранный вопрос: {probability_of_correct_answer:.2f}")
    
    current_ability = update_ability_estimate(model, X_new, np.array([y_new]), current_ability)
    print(f"Обновлённая оценка способностей: {current_ability:.2f}")  # Вывод обновлённой оценки способностей

    if len(available_questions) == 0:
        break

Выбран вопрос с ID: 57
Текст вопроса: Вопрос 57
Вероятность правильного ответа на выбранный вопрос: 0.81
Обновлённая оценка способностей: 0.42
Выбран вопрос с ID: 86
Текст вопроса: Вопрос 86
Вероятность правильного ответа на выбранный вопрос: 0.81
Обновлённая оценка способностей: 0.34
Выбран вопрос с ID: 47
Текст вопроса: Вопрос 47
Вероятность правильного ответа на выбранный вопрос: 0.81
Обновлённая оценка способностей: 0.36
Выбран вопрос с ID: 44
Текст вопроса: Вопрос 44
Вероятность правильного ответа на выбранный вопрос: 0.81
Обновлённая оценка способностей: 0.28
Выбран вопрос с ID: 98
Текст вопроса: Вопрос 98
Вероятность правильного ответа на выбранный вопрос: 0.81
Обновлённая оценка способностей: 0.30
Выбран вопрос с ID: 95
Текст вопроса: Вопрос 95
Вероятность правильного ответа на выбранный вопрос: 0.81
Обновлённая оценка способностей: 0.21
Выбран вопрос с ID: 21
Текст вопроса: Вопрос 21
Вероятность правильного ответа на выбранный вопрос: 0.81
Обновлённая оценка способностей: 0.13

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9349f80f-e3a6-4f6f-8e4c-025313d5eb81' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>